In [ ]:
import sys
sys.path.append('../..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from definitions import RESULTS_FOLDER, FIGURE_FOLDER

figure_location = os.path.join(FIGURE_FOLDER,'twostep')
if not os.path.exists(figure_location):
    os.makedirs(figure_location)

In [ ]:
data_lesion_hpc = pd.read_csv(os.path.join(RESULTS_FOLDER, 'twostep', 'results_lesion_hpc.csv'))
data_lesion_dls = pd.read_csv(os.path.join(RESULTS_FOLDER, 'twostep', 'results_lesion_dls.csv'))
data_control = pd.read_csv(os.path.join(RESULTS_FOLDER, 'twostep', 'results_control.csv'))

In [ ]:
data_lesion_dls.head()

In [ ]:
def is_common_or_rare(action, out):
    left_outcomes = (5, 6)
    right_outcomes = (7, 8)
    if action == 0 and out in left_outcomes:
        return 'common'
    elif action == 0 and out in right_outcomes:
        return 'rare'
    elif action == 1 and out in left_outcomes:
        return 'rare'
    elif action == 1 and out in right_outcomes:
        return 'common'
    else:
        raise ValueError('The combination of action and outcome does not make sense')

In [ ]:
def add_relevant_columns(dataframe):
    dataframe['PreviousAction'] = dataframe.groupby(['Agent'])['Action1'].shift(1)
    dataframe['PreviousStart'] = dataframe.groupby(['Agent'])['StartState'].shift(1)
    dataframe['PreviousReward'] = dataframe.groupby(['Agent'])['Reward'].shift(1)
    dataframe['Stay'] = (dataframe.PreviousAction == dataframe.Action1)
    dataframe['Transition'] = np.vectorize(is_common_or_rare)(dataframe['Action1'], dataframe['Terminus'])
    dataframe['PreviousTransition'] = dataframe.groupby(['Agent'])['Transition'].shift(1)
    

In [ ]:
add_relevant_columns(data_control)
add_relevant_columns(data_lesion_dls)
add_relevant_columns(data_lesion_hpc)

In [ ]:
data_lesion_dls.head()

In [ ]:
def compute_mean_stay_prob(data):
    means = data[data['Trial']>0].groupby(['PreviousTransition', 'PreviousReward'])['Stay'].mean()
    sems = data.groupby(['PreviousTransition', 'PreviousReward'])['Stay'].sem()
    return means, sems

In [ ]:
mean_lesion_hpc, sem_lesion_hpc = compute_mean_stay_prob(data_lesion_hpc)
mean_lesion_dls, sem_lesion_dls = compute_mean_stay_prob(data_lesion_dls)
mean_full, sem_full = compute_mean_stay_prob(data_control)

In [ ]:
mean_lesion_hpc

In [ ]:
def plot_daw_style(ax, data, yerr=None, title=''):
    lightgray = '#d1d1d1'
    darkgray = '#929292'

    bar_width= 0.2

    bars1 = data[:2][::-1]
    bars2 = data[2:][::-1]
    if yerr is not None:
        errs1 = yerr[:2][::-1]
        errs2 = yerr[2:][::-1]
    else:
        errs1 = yerr
        errs2 = yerr
        
    # The x position of bars
    r1 = np.array([0.125, 0.625]) 
    r2 = [x + bar_width + .05 for x in r1]
    list(sem_full),
    plt.sca(ax)
    
    plt.bar(r1, bars1, width=bar_width, color='blue',  capsize=4)
    plt.bar(r2, bars2, width=bar_width, color='red',  capsize=4)
    plt.xticks([r+ bar_width/2 +.025 for r in r1], ['Rewarded', 'Unrewarded'], fontsize=12)
    plt.yticks(fontsize=12)
    plt.title(title, fontsize=16)
    plt.ylim([0.4, 1])
    plt.xlim([0, 1])

In [ ]:
fig, axes = plt.subplots(1,3, figsize= (10,2.5), sharey=True)

plot_daw_style(axes[0], list(mean_lesion_hpc),  title='Striatum')
plot_daw_style(axes[1], list(mean_lesion_dls), title='Hippocampus')
plot_daw_style(axes[2], list(mean_full),  title='Full model')



leg = axes[0].legend(['Common', 'Rare'], fontsize=12, frameon=False, handlelength=0.7, title='Previous transition')
plt.sca(axes[0])
plt.ylabel('Stay probability', fontsize=12)

plt.savefig(os.path.join(figure_location, 'StayProbability.pdf'))

plt.show()

In [ ]:
plt.close()

# Doll et al analysis

In [ ]:
figure_location = os.path.join(FIGURE_FOLDER,'twostep_deterministic')
if not os.path.exists(figure_location):
    os.makedirs(figure_location)

# load data
data_lesion_hpc = pd.read_csv(os.path.join(RESULTS_FOLDER, 'twostep_deterministic', 'results_lesion_hpc.csv'))
data_lesion_dls = pd.read_csv(os.path.join(RESULTS_FOLDER, 'twostep_deterministic', 'results_lesion_dls.csv'))
data_control = pd.read_csv(os.path.join(RESULTS_FOLDER, 'twostep_deterministic', 'results_control.csv'))


In [ ]:
data_control.head()

In [ ]:
def add_relevant_columns(dataframe):
    dataframe['PreviousAction'] = dataframe.groupby(['Agent'])['Action1'].shift(1)
    dataframe['PreviousStart'] = dataframe.groupby(['Agent'])['StartState'].shift(1)
    dataframe['PreviousReward'] = dataframe.groupby(['Agent'])['Reward'].shift(1)
    dataframe['Stay'] = (dataframe.PreviousAction == dataframe.Action1)
    dataframe['SameStart'] = (dataframe.StartState == dataframe.PreviousStart)


add_relevant_columns(data_control)
add_relevant_columns(data_lesion_dls)
add_relevant_columns(data_lesion_hpc)

In [ ]:
def compute_mean_stay_prob(data):
    means = data[data['Trial']>0].groupby(['PreviousReward', 'SameStart'])['Stay'].mean()
    sems = data.groupby(['PreviousReward', 'SameStart'])['Stay'].sem()
    return means[::-1], sems[::-1]


In [ ]:
mean_lesion_hpc, sem_lesion_hpc = compute_mean_stay_prob(data_lesion_hpc)
mean_lesion_dls, sem_lesion_dls = compute_mean_stay_prob(data_lesion_dls)
mean_full, sem_full = compute_mean_stay_prob(data_control)

In [ ]:
mean_lesion_dls

In [ ]:
def plot_doll_style(ax, data, yerr=None, title=''):
    lightgray = '#d1d1d1'
    darkgray = '#929292'

    bar_width = 0.2

    bars1 = data[:2]
    bars2 = data[2:]
    if yerr is not None:
        errs1 = yerr[:2]
        errs2 = yerr[2:]
    else:
        errs1 = None
        errs2 = None 
        
    # The x position of bars
    r1 = np.arange(len(bars1)) * .8 + 1.5 * bar_width
    r2 = [x + bar_width for x in r1]

    plt.sca(ax)

    plt.bar(r1, bars1, width=bar_width, color=lightgray, yerr=errs1, capsize=4)
    plt.bar(r2, bars2, width=bar_width, color=darkgray, yerr=errs2, capsize=4)
    plt.ylabel('Stay probability', fontsize=15)
    plt.xticks([r + bar_width / 2 for r in r1], ['same', 'different'], fontsize=15)
    plt.yticks(fontsize=15)
    plt.title(title, fontsize=18)
    plt.ylim([0.40, .96])
    plt.xlim([0, 1.6])

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


In [ ]:
fig, axes = plt.subplots(1,3, figsize= (6,2.5), sharey=True)

plot_doll_style(axes[0], list(mean_lesion_hpc), title='Striatum')
plot_doll_style(axes[1], list(mean_lesion_dls), title='Hippocampus')
plot_doll_style(axes[2], list(mean_full),  title='Full model')



leg = axes[2].legend(['Reward', 'No reward'], fontsize=12, frameon=False, handlelength=.7)
leg.set_title('Previous outcome', prop={'size': 12})
plt.tight_layout()

plt.savefig(os.path.join(figure_location, 'StayProbability_DeterministicTask.pdf'))

plt.show()

In [ ]:
plt.close()